## Setup

Load libraries

In [1]:
import sys

sys.path.append('..')

In [2]:
import os
import pickle
from snowmodels.swe import HillSWE
from snowmodels.density import SturmDensity, JonasDensity, PistochiDensity 

/home/habeeb/miniconda3/envs/ml_density_env/lib/python3.11/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


## Load Dataset

In [3]:
with open('../data/data_splits.pkl', 'rb') as f:
    data_splits = pickle.load(f)

## Statistical Models

### Density Prediction (Test Set)

In [4]:
data_splits.y_test

0          0.308333
1          0.264286
2          0.300000
3          0.300000
4          0.278571
             ...   
2713944    0.300000
2713945    0.300000
2713946    0.300000
2713947    0.300000
2713948    0.300000
Name: Snow_Density, Length: 534818, dtype: float64

In [5]:
density_results=(
    data_splits.X_test
    .assign(
        Observed_Density=data_splits.y_test,
        Jonas_Density=lambda x: x.loc[:, ('Snow_Depth', 'Date', 'Elevation')].apply(lambda y: JonasDensity().compute_density(snow_depth=y.Snow_Depth*0.01, month=y.Date.month_name(), elevation= y.Elevation), axis=1),
        Pistochi_Density=lambda x: x.loc[:, 'Pistochi_DOWY'].apply(lambda y: PistochiDensity().compute_density(DOY=y)),
        Sturm_Density=lambda x: x.loc[:, ('Snow_Depth', 'Sturm_DOWY', 'Snow_Class')].apply(lambda y: SturmDensity().compute_density(snow_depth=y.Snow_Depth, DOY=y.Sturm_DOWY, snow_class= y.Snow_Class), axis=1) 
    )
    .filter(items=['Snow_Class', 'Observed_Density', 'Jonas_Density', 'Pistochi_Density', 'Sturm_Density'])
    .reset_index(drop=True)
)

density_results.head()

,Snow_Class,Observed_Density,Jonas_Density,Pistochi_Density,Sturm_Density
0,Prairie,0.308333,0.222326,0.323,0.251437
1,Prairie,0.264286,0.224713,0.324,0.255262
2,Prairie,0.300000,0.223519,0.325,0.254935
3,Prairie,0.300000,0.223519,0.326,0.255984
4,Prairie,0.278571,0.224713,0.327,0.258397


### SWE Prediction (Test Set)

In [6]:
swe_results=(
    density_results
    .assign(
        Observed_SWE=data_splits.X_test.reset_index().SWE,
        Snow_Depth=data_splits.X_test.reset_index().Snow_Depth,
        Jonas_SWE=lambda x: x.Jonas_Density * x.Snow_Depth,
        Sturm_SWE=lambda x: x.Sturm_Density * x.Snow_Depth,
        Pistochi_SWE=lambda x: x.Pistochi_Density * x.Snow_Depth,
        Hill_SWE=data_splits.X_test.reset_index()[['PPTWT', 'Temp_Diff', 'DOWY', 'Snow_Depth']].apply(lambda x: HillSWE().compute_swe(pptwt=x.PPTWT, TD=x.Temp_Diff, DOY=x.DOWY, snow_depth=x.Snow_Depth*10), axis=1)
    )
    .drop(
        columns=[
            'Observed_Density', 
            'Jonas_Density', 
            'Pistochi_Density', 
            'Sturm_Density', 
            'Snow_Depth',
        ]
    )
)

swe_results.head()

,Snow_Class,Observed_SWE,Jonas_SWE,Sturm_SWE,Pistochi_SWE,Hill_SWE
0,Prairie,9.398,6.776484,7.663798,9.84504,NaN
1,Prairie,9.398,7.990801,9.077107,11.52144,NaN
2,Prairie,9.906,7.380611,8.417954,10.73150,NaN
3,Prairie,9.906,7.380611,8.452608,10.76452,NaN
4,Prairie,9.906,7.990801,9.188611,11.62812,NaN


In [7]:
from sklearn.metrics import root_mean_squared_error, r2_score


root_mean_squared_error(y_true=density_results.dropna().Observed_Density, y_pred=density_results.dropna().Jonas_Density), r2_score(y_true=density_results.dropna().Observed_Density, y_pred=density_results.dropna().Jonas_Density)

(0.06596123378916076, 0.44704173149188375)

## Store Predictions to File

In [8]:
os.makedirs('../results', exist_ok=True)
os.makedirs('../results/density', exist_ok=True)
os.makedirs('../results/swe', exist_ok=True)

density_results.to_parquet('../results/density/statistical_models_predictions.parquet', index=False, compression='gzip')
swe_results.to_parquet('../results/swe/statistical_models_predictions.parquet', index=False, compression='gzip')

## The End!